In [15]:
from model import CharRNN
import tensorflow as tf
import numpy as np
import time
import string

batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability
epochs = 20
print_every_n = 100
save_every_n = 200

# Read text
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.lower()
    text = "".join(filter(lambda x: x in string.printable, text))
    
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

print("Read %s length text with vocabulary of: %s" % (len(text), len(vocab)))



Read 21102610 length text with vocabulary of: 72


In [16]:
# Get Batches
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    chars_per_batch = batch_size * n_steps
    n_batches = len(arr) // chars_per_batch

    arr = arr[:n_batches * chars_per_batch]

    arr = arr.reshape((batch_size, -1))

    for n in range(0, arr.shape[1], n_steps):
        x = arr[:, n:n + n_steps]
        y_temp = arr[:, n + 1:n + n_steps + 1]
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:, :y_temp.shape[1]] = y_temp

        yield x, y


In [17]:
# Train Model

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers,
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    counter = 0
    for e in range(epochs):
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state,  _ = sess.run([model.loss,
                                                  model.final_state,
                                                  model.optimizer],
                                                 feed_dict=feed)

            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e + 1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end - start)))

            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 100...  Training loss: 2.9903...  0.3047 sec/batch
Epoch: 1/20...  Training Step: 200...  Training loss: 2.4421...  0.3056 sec/batch
Epoch: 1/20...  Training Step: 300...  Training loss: 2.2373...  0.3058 sec/batch
Epoch: 1/20...  Training Step: 400...  Training loss: 2.0642...  0.3056 sec/batch
Epoch: 1/20...  Training Step: 500...  Training loss: 1.9435...  0.3058 sec/batch
Epoch: 1/20...  Training Step: 600...  Training loss: 1.8561...  0.3057 sec/batch
Epoch: 1/20...  Training Step: 700...  Training loss: 1.8078...  0.3059 sec/batch
Epoch: 1/20...  Training Step: 800...  Training loss: 1.7450...  0.3064 sec/batch
Epoch: 1/20...  Training Step: 900...  Training loss: 1.6692...  0.3053 sec/batch
Epoch: 1/20...  Training Step: 1000...  Training loss: 1.6491...  0.3061 sec/batch
Epoch: 1/20...  Training Step: 1100...  Training loss: 1.6070...  0.3051 sec/batch
Epoch: 1/20...  Training Step: 1200...  Training loss: 1.6719...  0.3058 sec/batch
Epoch: 1/20..

Epoch: 5/20...  Training Step: 10000...  Training loss: 1.2531...  0.3061 sec/batch
Epoch: 5/20...  Training Step: 10100...  Training loss: 1.2980...  0.3059 sec/batch
Epoch: 5/20...  Training Step: 10200...  Training loss: 1.3279...  0.3060 sec/batch
Epoch: 5/20...  Training Step: 10300...  Training loss: 1.2704...  0.3066 sec/batch
Epoch: 5/20...  Training Step: 10400...  Training loss: 1.2448...  0.3065 sec/batch
Epoch: 5/20...  Training Step: 10500...  Training loss: 1.2523...  0.3054 sec/batch
Epoch: 6/20...  Training Step: 10600...  Training loss: 1.3056...  0.3061 sec/batch
Epoch: 6/20...  Training Step: 10700...  Training loss: 1.2804...  0.3054 sec/batch
Epoch: 6/20...  Training Step: 10800...  Training loss: 1.2678...  0.3072 sec/batch
Epoch: 6/20...  Training Step: 10900...  Training loss: 1.2402...  0.3066 sec/batch
Epoch: 6/20...  Training Step: 11000...  Training loss: 1.2749...  0.3063 sec/batch
Epoch: 6/20...  Training Step: 11100...  Training loss: 1.2682...  0.3056 se

Epoch: 10/20...  Training Step: 19800...  Training loss: 1.2352...  0.3072 sec/batch
Epoch: 10/20...  Training Step: 19900...  Training loss: 1.2300...  0.3058 sec/batch
Epoch: 10/20...  Training Step: 20000...  Training loss: 1.2204...  0.3069 sec/batch
Epoch: 10/20...  Training Step: 20100...  Training loss: 1.2323...  0.3063 sec/batch
Epoch: 10/20...  Training Step: 20200...  Training loss: 1.2184...  0.3056 sec/batch
Epoch: 10/20...  Training Step: 20300...  Training loss: 1.1687...  0.3055 sec/batch
Epoch: 10/20...  Training Step: 20400...  Training loss: 1.2148...  0.3063 sec/batch
Epoch: 10/20...  Training Step: 20500...  Training loss: 1.2280...  0.3065 sec/batch
Epoch: 10/20...  Training Step: 20600...  Training loss: 1.2194...  0.3061 sec/batch
Epoch: 10/20...  Training Step: 20700...  Training loss: 1.2448...  0.3063 sec/batch
Epoch: 10/20...  Training Step: 20800...  Training loss: 1.2560...  0.3057 sec/batch
Epoch: 10/20...  Training Step: 20900...  Training loss: 1.2237..

Epoch: 14/20...  Training Step: 29500...  Training loss: 1.2105...  0.3057 sec/batch
Epoch: 15/20...  Training Step: 29600...  Training loss: 1.1834...  0.3056 sec/batch
Epoch: 15/20...  Training Step: 29700...  Training loss: 1.2034...  0.3067 sec/batch
Epoch: 15/20...  Training Step: 29800...  Training loss: 1.2421...  0.3064 sec/batch
Epoch: 15/20...  Training Step: 29900...  Training loss: 1.2304...  0.3064 sec/batch
Epoch: 15/20...  Training Step: 30000...  Training loss: 1.2019...  0.3058 sec/batch
Epoch: 15/20...  Training Step: 30100...  Training loss: 1.2211...  0.3055 sec/batch
Epoch: 15/20...  Training Step: 30200...  Training loss: 1.2211...  0.3056 sec/batch
Epoch: 15/20...  Training Step: 30300...  Training loss: 1.1866...  0.3065 sec/batch
Epoch: 15/20...  Training Step: 30400...  Training loss: 1.1887...  0.3060 sec/batch
Epoch: 15/20...  Training Step: 30500...  Training loss: 1.1720...  0.3068 sec/batch
Epoch: 15/20...  Training Step: 30600...  Training loss: 1.2281..

KeyboardInterrupt: 

In [18]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i33600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i13800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i14000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i14200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i14400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i14600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i14800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i15000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i15200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i15400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i15600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i15800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i16000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i16200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i16400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i16600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i16800_l512.ckpt"
al

In [19]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [20]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [21]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i33600_l512.ckpt'

In [26]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="poop")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i33600_l512.ckpt
poop are the only reason i want to store
hep: loool
ryan: i wonder what his children is back
rene: lol
david: i have no idea what to think is i wont be free in a long time
hep: lol
hep: i have to go away
david: then we are already sours but not in an explusion in the face
ryan: the campun all sense is about sami well i dont want to be a last work
samantha: it's a little bad at this thing
ryan: they did today they dont have a couple days out i did
samantha: i'd have a little stuff
samantha: sorry if i was the born we will hung out i can't even die in my building i think
ryan: i want to see those
ryan: thong is still the one that helps
ryan: she didnt, i was like "yeah i'm going to go at this bed this minute, and his brother's answer then they didn't like them"
samantha: i thought hidestays are sure tho
david: lol
ryan: i didnt
ryan: thats not supposed to be this time
david: it should just be famous
ryan: also i think